In [ ]:
import numpy as np
from scipy.interpolate import make_interp_spline
from scipy.ndimage import gaussian_filter1d
from sklearn.preprocessing import MinMaxScaler
import os
import joblib
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, LayerNormalization, MultiHeadAttention, Dropout, Layer, Flatten, GlobalAveragePooling1D
from tensorflow.keras.models import Model
from sklearn.model_selection import train_test_split

In [ ]:
# Constants
mu_0 = 4 * np.pi * 1e-7  # Permeability of free space (H/m)
frequencies = np.array([1.000000e+04, 8.799998e+03, 7.200000e+03, 6.000000e+03, 5.200001e+03,  4.400000e+03, 3.600000e+03, 3.000001e+03, 2.600000e+03, 2.200000e+03,  1.800000e+03, 1.500000e+03, 1.300000e+03, 1.058824e+03, 9.176473e+02,  7.764706e+02, 6.352943e+02, 5.294117e+02, 4.588235e+02, 3.882354e+02,  3.176470e+02, 2.647059e+02, 2.294118e+02, 1.941176e+02, 1.588235e+02,  1.323529e+02, 1.147059e+02, 9.705883e+01, 7.941176e+01, 6.499999e+01,  5.499999e+01, 3.750000e+01, 3.250000e+01, 2.750000e+01, 2.250000e+01,  1.875000e+01, 1.625000e+01, 1.375000e+01, 1.125000e+01, 9.375000e+00,  8.125000e+00, 6.875000e+00, 5.625000e+00, 4.687500e+00, 4.062500e+00,  3.437500e+00, 2.812500e+00, 2.343750e+00, 1.718750e+00, 1.406250e+00,  1.171875e+00, 1.015625e+00, 8.593750e-01, 7.031250e-01, 5.859375e-01,  5.078125e-01, 4.296875e-01, 3.515625e-01, 2.929688e-01, 2.539063e-01,  1.757813e-01, 8.789063e-02, 5.371094e-02, 4.394531e-02, 3.662109e-02,  3.173828e-02, 2.685547e-02, 2.197266e-02, 1.831055e-02, 1.586914e-02,  1.342773e-02, 1.098633e-02, 9.155274e-03, 7.934572e-03, 6.713867e-03,  5.493165e-03, 4.577638e-03, 3.967284e-03, 3.356934e-03, 2.746581e-03,  2.288818e-03, 1.983643e-03, 1.678467e-03, 1.373291e-03, 1.144409e-03,  9.918214e-04, 8.392333e-04, 6.866456e-04, 5.722047e-04, 3.433228e-04])
periods = 1 / frequencies  # Periods in seconds
resistivity_range = [1, 500]  # Min and max resistivities in Ohm.m
rho_ref = 50  # Reference resistivity in Ohm.m
num_layers = 50

# Generate depths using an initial value and multiplying by a factor
depths = [8]  # Starting depth
for i in range(num_layers - 1):
    depth = depths[-1] * 1.2078  #We want to get 50 depth points ending at about 100,000
    depths.append(depth)


# Calculate layer thicknesses from depths
layer_thicknesses = np.diff(depths)  # Difference between consecutive depths
layer_thicknesses = np.insert(layer_thicknesses, 0, depths[0])  # Insert first depth as the initial thickness


# Function to generate resistivity profile based on depths
def generate_smooth_resistivity_profile(depths, resistivity_range, num_spline_points, smoothing=True):
    """
    Generates a smooth resistivity profile where resistivities align with depth points.

    Parameters:
    - depths: Array of depth points for the layers
    - resistivity_range: List with min and max resistivity values
    - num_spline_points: Number of points to generate spline (int)
    - smoothing: Whether to apply Gaussian smoothing (bool)

    Returns:
    - resistivities: Array of resistivity values aligned with each layer's depth
    """
    # Generate random spline points between the minimum and maximum depths
    spline_points_depth = np.linspace(depths[0], depths[-1], num=num_spline_points)
    resistivity_points = np.random.uniform(resistivity_range[0], resistivity_range[1], num_spline_points)

    # Create cubic spline for interpolation
    spline = make_interp_spline(spline_points_depth, resistivity_points, k=3)

    # Interpolate resistivities for the given depths
    resistivities = spline(depths)

    if smoothing:
        # Apply Gaussian smoothing to resistivities to reduce sharp transitions
        resistivities = gaussian_filter1d(resistivities, sigma=1)

    return resistivities


# Compute apparent resistivity and phase
def compute_apparent_resistivity_and_phase(thicknesses, conductivities, periods):
    apparent_resistivity = []
    phase = []

    for T in periods:
        omega = 2 * np.pi / T  # Angular frequency (rad/s)
        cns = np.zeros(len(conductivities), dtype=complex)
        cns[-1] = 1 / np.sqrt(mu_0 * omega * conductivities[-1] * 1j)

        for j in reversed(range(len(thicknesses))):
            K = np.sqrt(conductivities[j] * mu_0 * omega * 1j)
            layer_thickness = thicknesses[j] if j < len(thicknesses) - 1 else np.inf
            if j + 1 < len(cns):
                cns[j] = (1 / K) * ((K * cns[j + 1] + np.tanh(K * layer_thickness)) /
                                    (1 + K * cns[j + 1] * np.tanh(K * layer_thickness)))

        Z = cns[0]
        rho_apparent = np.abs(Z) ** 2 / (mu_0 * omega)
        phi = np.angle(Z, deg=True) + 90

        apparent_resistivity.append(rho_apparent)
        phase.append(phi)

    return np.array(apparent_resistivity), np.array(phase)



num_spline_points_list = [6, 7, 8, 9, 10]  # Number of spline points
examples_per_spline = 20000  # Number of examples per spline point count
total_examples = examples_per_spline * len(num_spline_points_list)  # Total examples: 100,000

X_model = np.empty((total_examples, num_layers))
y_rho = np.empty((total_examples, len(periods)))
y_phi = np.empty((total_examples, len(periods)))

example_idx = 0  # Index to keep track of the current example

#Generate data
for num_spline_points in num_spline_points_list:
    print(f"Generating {examples_per_spline} examples with {num_spline_points} spline points...")
    for _ in range(examples_per_spline):
        # Generate smooth resistivity profile
        resistivities = generate_smooth_resistivity_profile(
            depths=depths,
            resistivity_range=resistivity_range,
            num_spline_points=num_spline_points,
            smoothing=True
        )
        conductivities = 1 / resistivities
        apparent_resistivity, phase = compute_apparent_resistivity_and_phase(
            layer_thicknesses,
            conductivities,
            periods
        )

        # Store the results
        X_model[example_idx] = resistivities
        y_rho[example_idx] = apparent_resistivity
        y_phi[example_idx] = phase
        example_idx += 1

    print(f"Completed {examples_per_spline} examples with {num_spline_points} spline points.")

print("Data generation completed.")

X_model = np.array(X_model)
y_rho = np.array(y_rho)
y_phi = np.array(y_phi)

# Normalize data
scaler_X_model = MinMaxScaler()
scaler_y_rho = MinMaxScaler()
scaler_y_phi = MinMaxScaler()

scaler_X_model.fit(X_model)
scaler_y_rho.fit(y_rho)
scaler_y_phi.fit(y_phi)
X_model_scaled = scaler_X_model.transform(X_model)
y_rho_scaled = scaler_y_rho.transform(y_rho)
y_phi_scaled = scaler_y_phi.transform(y_phi)

# Combine y_rho and y_phi for the input data (each frequency has rho and phi as components)
X_combined = np.stack((y_rho_scaled, y_phi_scaled), axis=-1)  # Shape: (num_stations, num_periods, 2)

save_path = ""#Tha path to save the data

# Save the generated data to Google Drive
np.save(save_path + 'X_combined_u.npy', X_combined)
np.save(save_path + 'X_model_scaled_u.npy', X_model_scaled)

# Save the scalers
joblib.dump(scaler_X_model, save_path + 'scaler_X_model_u.pkl')
joblib.dump(scaler_y_rho, save_path + 'scaler_y_rho_u.pkl')
joblib.dump(scaler_y_phi, save_path + 'scaler_y_phi_u.pkl')

In [ ]:
load_path = "" #Path to where the data is saved
X_combined = np.load(os.path.join(load_path, 'X_combined2.npy'))  # Shape: (100000, 90, 2)
X_model_scaled = np.load(os.path.join(load_path, 'X_model_scaled2.npy'))  # Shape: (100000, 100)
print("Data loaded successfully.")


class TransformerAttentionBlock(Layer):
    def __init__(self, embed_dim, num_heads, ff_dim, dropout_rate=0.01):
        super(TransformerAttentionBlock, self).__init__()
        self.attention = MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)
        self.ffn = tf.keras.Sequential([
            Dense(ff_dim, activation="relu"),  # Feedforward network
            Dense(embed_dim)
        ])
        self.layernorm1 = LayerNormalization(epsilon=1e-6)
        self.layernorm2 = LayerNormalization(epsilon=1e-6)
        self.dropout1 = Dropout(dropout_rate)
        self.dropout2 = Dropout(dropout_rate)

    def call(self, inputs, training=False):
        # Self-attention
        attn_output = self.attention(inputs, inputs)
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.layernorm1(inputs + attn_output)

        # Feedforward network
        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output, training=training)
        return self.layernorm2(out1 + ffn_output)


# Graph Attention Layer
class GraphAttentionLayer(Layer):
    def __init__(self, output_dim, num_heads=4, concat=False, dropout_rate=0.01, activation=None, **kwargs):
        super(GraphAttentionLayer, self).__init__(**kwargs)
        self.output_dim = output_dim
        self.num_heads = num_heads
        self.concat = concat
        self.dropout_rate = dropout_rate
        self.activation = activation

    def build(self, input_shape):
        input_dim = input_shape[0][-1]
        self.W = self.add_weight(shape=(input_dim, self.num_heads * self.output_dim),
                                 initializer='glorot_uniform', name='W')
        self.a_src = self.add_weight(shape=(self.num_heads, self.output_dim, 1),
                                     initializer='glorot_uniform', name='a_src')
        self.a_dst = self.add_weight(shape=(self.num_heads, self.output_dim, 1),
                                     initializer='glorot_uniform', name='a_dst')
        super(GraphAttentionLayer, self).build(input_shape)

    def call(self, inputs, training=False):
        X, adjacency = inputs
        batch_size = tf.shape(X)[0]
        num_nodes = tf.shape(X)[1]

        XW = tf.matmul(X, self.W)
        XW = tf.reshape(XW, (batch_size, num_nodes, self.num_heads, self.output_dim))
        XW = tf.transpose(XW, perm=[0, 2, 1, 3])

        e_src = tf.einsum('hdf,bhnd->bhn', self.a_src, XW)
        e_dst = tf.einsum('hdf,bhnd->bhn', self.a_dst, XW)

        e_src_expanded = tf.expand_dims(e_src, axis=-1)
        e_dst_expanded = tf.expand_dims(e_dst, axis=2)
        e = e_src_expanded + e_dst_expanded

        e = tf.nn.leaky_relu(e, alpha=0.2)
        mask = adjacency > 0
        mask = tf.expand_dims(mask, axis=1)
        e = tf.where(mask, e, tf.fill(tf.shape(e), -1e9))

        alpha = tf.nn.softmax(e, axis=-1)
        alpha = tf.nn.dropout(alpha, rate=self.dropout_rate) if training else alpha
        out = tf.matmul(alpha, XW)

        if self.concat:
            out = tf.transpose(out, perm=[0, 2, 1, 3])
            out = tf.reshape(out, (batch_size, num_nodes, self.num_heads * self.output_dim))
        else:
            out = tf.reduce_mean(out, axis=1)

        if self.activation:
            out = self.activation(out)
        return out


# GAT Model with Two Attention Blocks and Dense Layers Afterward
class GAT(Model):
    def __init__(self, num_heads=8, hidden_dim=64, output_dim=105, dropout_rate=0.1, **kwargs):
        super(GAT, self).__init__(**kwargs)
        self.gat_output_dim = output_dim
        self.dropout_rate = dropout_rate
        self.gat1 = GraphAttentionLayer(output_dim=hidden_dim, num_heads=num_heads, concat=True,
                                        dropout_rate=dropout_rate, activation=tf.nn.elu)
        self.gat2 = GraphAttentionLayer(output_dim=hidden_dim, num_heads=1, concat=False,
                                        dropout_rate=dropout_rate, activation=tf.nn.elu)
        # Global Pooling layer for graph-level output
        self.global_pool = GlobalAveragePooling1D()

        # Instantiate two TransformerAttentionBlocks
        self.transformer_attention1 = TransformerAttentionBlock(embed_dim=16, num_heads=4, ff_dim=64, dropout_rate=0.01)
        self.transformer_attention2 = TransformerAttentionBlock(embed_dim=16, num_heads=4, ff_dim=64, dropout_rate=0.01)

        # Define dense layers after attention blocks
        self.dense1 = Dense(1024)
        self.dense2 = Dense(1024)
        self.final_dense = Dense(output_dim)

    def call(self, inputs, training=False):
        X, adjacency = inputs
        X = tf.nn.dropout(X, rate=self.dropout_rate) if training else X
        X = self.gat1((X, adjacency), training=training)
        X = tf.nn.dropout(X, rate=self.dropout_rate) if training else X
        X = self.gat2((X, adjacency), training=training)
        X = self.global_pool(X)  # Shape: (batch_size, hidden_dim)

        # Reshape for attention blocks
        reshaped_output = tf.reshape(X, (-1, 4, 16))  # Since hidden_dim=64, 4*16=64

        # Apply first attention block
        attention_output = self.transformer_attention1(reshaped_output, training=training)

        # Apply second attention block
        attention_output = self.transformer_attention2(attention_output, training=training)

        # Aggregate attention outputs
        aggregated_output = tf.reduce_mean(attention_output, axis=1)  # Shape: (batch_size, embed_dim)

        # Apply dense layers
        X = self.dense1(aggregated_output)
        X = self.dense2(X)
        # Final output
        X = self.final_dense(X)
        return X

# Adjacency Matrix
def create_adjacency_matrix(num_nodes=90, k=5):
    adj = np.zeros((num_nodes, num_nodes), dtype=np.float32)
    for i in range(num_nodes):
        for j in range(max(0, i - k), min(num_nodes, i + k + 1)):
            if i != j:
                adj[i, j] = 1.0
    np.fill_diagonal(adj, 1.0)
    return adj

adjacency_matrix = create_adjacency_matrix(num_nodes=90, k=5)

# Prepare Data
X = X_combined  # Node features
y = X_model_scaled  # Targets (graph-level outputs)

# Replicate adjacency matrix for all samples
num_samples = X.shape[0]
A = np.tile(adjacency_matrix, (num_samples, 1, 1))

# Split Dataset into Training and Validation Sets
X_train, X_val, A_train, A_val, y_train, y_val = train_test_split(
    X, A, y, test_size=0.1, random_state=42
)

# Convert to TensorFlow Tensors
X_train = tf.convert_to_tensor(X_train, dtype=tf.float32)
X_val = tf.convert_to_tensor(X_val, dtype=tf.float32)
A_train = tf.convert_to_tensor(A_train, dtype=tf.float32)
A_val = tf.convert_to_tensor(A_val, dtype=tf.float32)
y_train = tf.convert_to_tensor(y_train, dtype=tf.float32)
y_val = tf.convert_to_tensor(y_val, dtype=tf.float32)

buffer_size = X_train.shape[0]

# Create TensorFlow datasets
train_dataset = tf.data.Dataset.from_tensor_slices(((X_train, A_train), y_train)).shuffle(buffer_size).batch(64).prefetch(tf.data.AUTOTUNE)
val_dataset = tf.data.Dataset.from_tensor_slices(((X_val, A_val), y_val)).batch(64).prefetch(tf.data.AUTOTUNE)

print("Training and validation datasets prepared.")

# Define and Train the GAT Model
gat_model = GAT(num_heads=8, hidden_dim=64, output_dim=100, dropout_rate=0.01)

# Compile the Model
gat_model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
    loss='mean_squared_error',
    metrics=['mae']
)

# Build the Model
node_features_input = Input(shape=(90, 2), dtype=tf.float32)
adjacency_input = Input(shape=(90, 90), dtype=tf.float32)
gat_output = gat_model((node_features_input, adjacency_input))

# Train the Model
history = gat_model.fit(train_dataset, validation_data=val_dataset, epochs=100)
